In [1]:
import os
from PIL import Image
import numpy as np
pred_list = os.listdir('glas_ours_testPseudoMask')
inter_union = np.zeros((2,2))
for pred_file in pred_list:
    pred = np.asarray(Image.open((f'glas_ours_testPseudoMask/{pred_file}')))
    gt = np.asarray(Image.open(f'Dataset_warwick/2.validation/mask/{pred_file}'))
    gt[gt > 0] = 1
    # gt[gt == 0] = 1
    # gt[gt == 3] = 0
    
    for i in range(2):
        if i in pred:
            inter = np.sum(np.logical_and(pred == i, gt == i))
            u = np.sum(np.logical_or(pred == i, gt == i))
            inter_union[0,i] += inter
            inter_union[1,i] += u

total = 0
eps = 1e-4
for i in range(2):
    class_i = inter_union[0,i] / (inter_union[1,i] + eps)
    total += class_i
mIOU = total / 2
dice = 2 * mIOU / (1 + mIOU)
print(mIOU, dice)

0.700375976447313 0.8237895455458577


In [9]:
import os
from PIL import Image
import numpy as np
pred_list = os.listdir('glas_ours_trainPseudoMask')
inter_union = np.zeros((2,2))
for pred_file in pred_list:
    pred = np.asarray(Image.open((f'glas_ours_trainPseudoMask/{pred_file}')))
    gt = np.asarray(Image.open(f'Dataset_warwick/1.training/mask/{pred_file[:-4]}_anno.bmp'))
    gt[gt > 0] = 3
    gt[gt == 0] = 1
    gt[gt == 3] = 0
    
    for i in range(2):
        if i in pred:
            inter = np.sum(np.logical_and(pred == i, gt == i))
            u = np.sum(np.logical_or(pred == i, gt == i))
            inter_union[0,i] += inter
            inter_union[1,i] += u

total = 0
eps = 1e-4
for i in range(2):
    class_i = inter_union[0,i] / (inter_union[1,i] + eps)
    total += class_i
mIOU = total / 2
dice = 2 * mIOU / (1 + mIOU)
print(mIOU, dice)

0.750178092726126 0.8572591507617694


In [ ]:
import os
from PIL import Image
import numpy as np
import png
pred_list = os.listdir('/home/yzouag2/WSSS4LUAD/glas_ours_trainPseudoMask')
inter_union = np.zeros((2,2))
destination = 'fuck'
if not os.path.exists(destination):
    os.mkdir(destination)

for pred_file in pred_list:
    gt = np.asarray(Image.open(f'Dataset_warwick/1.training/mask/{pred_file[:-4]}_anno.bmp'))
    gt[gt > 0] = 3
    gt[gt == 0] = 1
    gt[gt == 3] = 0
    palette = [(0, 64, 128), (64, 128, 0), (243, 152, 0), (255, 255, 255)]
    with open(os.path.join(destination, f'{pred_file[:-4]}_anno.png'), 'wb') as f:
        w = png.Writer(gt.shape[1], gt.shape[0],palette=palette, bitdepth=8)
        w.write(f, gt.astype(np.uint8))

In [2]:
from PIL import Image
import os
import numpy as np
from tqdm import tqdm

img_path = 'Dataset_warwick/2.validation/img'
mask_path = 'Dataset_warwick/2.validation/mask'

destination = 'cropImage'
destination1 = 'cropImageGT'
if not os.path.exists(destination):
    os.mkdir(destination)
if not os.path.exists(destination1):
    os.mkdir(destination1)

def online_cut_patches(im, im_size=96, stride=32):
    """
    function for crop the image to subpatches, will include corner cases
    the return position (x,y) is the up left corner of the image
    Args:
        im (np.ndarray): the image for cropping
        im_size (int, optional): the sub-image size. Defaults to 56.
        stride (int, optional): the pixels between two sub-images. Defaults to 28.
    Returns:
        (list, list): list of image reference and list of its corresponding positions
    """
    im_list = []
    position_list = []

    h, w, _ = im.shape
    if h < im_size:
        h_ = np.array([0])
    else:
        h_ = np.arange(0, h - im_size + 1, stride)
        if h % stride != 0:
            h_ = np.append(h_, h-im_size)

    if w < im_size:
        w_ = np.array([0])
    else:
        w_ = np.arange(0, w - im_size + 1, stride)
        if w % stride != 0:
            w_ = np.append(w_, w - im_size)

    for i in h_:
        for j in w_:   	
            temp = np.uint8(im[i:i+im_size,j:j+im_size,:])
            im_list.append(temp)
            position_list.append((i,j))
    return im_list, position_list

for i in tqdm(range(1, 81)):
    im = np.asarray(Image.open(os.path.join(img_path, f'{i}.bmp')))
    mask = np.asarray(Image.open(os.path.join(mask_path, f'{i}.png')))
    crop_list, positions_list = online_cut_patches(im, 224, 224)
    for j in range(len(crop_list)):
        crop_im = crop_list[j]
        position = positions_list[j]
        crop_mask = mask[position[0]:position[0]+224, position[1]:position[1]+224]
        has_tumor = 0
        has_normal = 0
        if np.sum(crop_mask > 0) / crop_mask.size > 0.05:
            has_tumor = 1
        if np.sum(crop_mask == 0) / crop_mask.size > 0.05:
            has_normal = 1
        Image.fromarray(crop_im).save(os.path.join(destination, f'train_{i}_{j}-[{has_tumor}, {has_normal}].png'))
        Image.fromarray(crop_mask).save(os.path.join(destination1, f'train_{i}_{j}-[{has_tumor}, {has_normal}].png'))
    

100%|██████████| 80/80 [00:15<00:00,  5.12it/s]


In [6]:
from PIL import Image
import os
import numpy as np
import png

a = np.asarray(Image.open('/home/yzouag2/WSSS4LUAD/Dataset_warwick/1.training/mask/train_1_anno.bmp'))
a[a >0] = 1
palette = [(0, 64, 128), (64, 128, 0), (243, 152, 0), (255, 255, 255)]
with open('test_train.png', 'wb') as f:
    w = png.Writer(a.shape[1], a.shape[0],palette=palette, bitdepth=8)
    w.write(f, a.astype(np.uint8))

In [9]:
from PIL import Image
import png
import numpy as np
import os

path = 'fully'
for file in os.listdir(path):
    a = np.asarray(Image.open(os.path.join(path, file)))
    palette = [(0, 64, 128), (64, 128, 0), (243, 152, 0), (255, 255, 255)]
    with open('fully/'+file, 'wb') as f:
        w = png.Writer(a.shape[1], a.shape[0],palette=palette, bitdepth=8)
        w.write(f, a.astype(np.uint8))

In [2]:
import numpy as np
a = np.array([[1,0,0],[1,0,1]])
b = np.array([[1,1,1],[0,0,1]])
TP = np.sum(np.logical_and(a == 0, b == 0))
FP = np.sum(np.logical_and(a == 0, b == 1))
FN = np.sum(np.logical_and(a == 1, b == 0))

In [7]:
import os
import numpy as np
from PIL import Image


paths = ['test_data/glas_ours_testPseudoMask', 'test_data/glas_seam_testPseudoMask', 'test_data/oeem_loss_merge', 'test_data/fully']
gt_path = 'test_data/gt'
for path in paths:
    TPs = 0
    FPs = 0
    FNs = 0
    ims = os.listdir(path)
    for im in ims:
        pred = np.asarray(Image.open(os.path.join(path, im)))
        gt = np.asarray(Image.open(os.path.join(gt_path, im)))
        TP = np.sum(np.logical_and(pred == 0, gt == 0)) # 0 is glandular, 1 is non-glandular
        FP = np.sum(np.logical_and(pred == 0, gt == 1))
        FN = np.sum(np.logical_and(pred == 1, gt == 0))
        TPs += TP
        FPs += FP
        FNs += FN
    f1_score = TPs / (TPs + (FPs + FNs)/2)
    print(f'{path}, F1 score: {f1_score}')

test_data/glas_ours_testPseudoMask, F1 score: 0.8376070680206164
test_data/glas_seam_testPseudoMask, F1 score: 0.7746162921781536
test_data/oeem_loss_merge, F1 score: 0.8816267849093171
test_data/fully, F1 score: 0.934429999873453


In [11]:
import numpy as np
from PIL import Image

gt_path = 'Dataset_warwick/2.validation/mask/8.png'
pred_path = 'glas_ours_testPseudoMask/8.png'

gt = np.asarray(Image.open(gt_path))
pred = np.asarray(Image.open(pred_path))

false_map = np.zeros((gt.shape[0], gt.shape[1], 4))
false_map[gt == pred] = [255, 255, 255, 0]
false_map[gt != pred] = [0, 0, 255, 150]

Image.fromarray(false_map.astype(np.uint8)).save('test.png')